<a href="https://colab.research.google.com/github/1kaiser/ICESAT2/blob/main/ICESAT2_tracks_intersection_plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# prerequisite


#[**Work Book**](docs.google.com/spreadsheets/d/1bYvEQ2ckp-xSrT_Xsa-lAnW3MEQU9K8AnXNjGL81C4g) **for processing links from earth data cloud ALT03 tracks in the region of ICESAT 2**

In [ ]:


# list_B = [
# "https://n5eil02u.ecs.nsidc.org/esir/500000420137/22297319/processed_ATL03_20181027214446_04470106_005_01.h5",
# "https://n5eil02u.ecs.nsidc.org/esir/500000420137/22319537/processed_ATL03_20181106091229_05920102_005_01.h5",
# "https://n5eil02u.ecs.nsidc.org/esir/500000420137/22325051/processed_ATL03_20181125202044_08890106_005_01.h5",
# "https://n5eil02u.ecs.nsidc.org/esir/500000420137/22343676/processed_ATL03_20181224185651_13310106_005_01.h5",
# "https://n5eil02u.ecs.nsidc.org/esir/500000420137/22346036/processed_ATL03_20190107061605_01500202_005_01.h5",
# ]



# for i in list_B:
#   !wget --http-user=kroy0001 --http-password=/#j%kWrPA,8.HRe {i}

In [ ]:
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/ASTER_DEM.tif #getting the dem
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/pongdam_dem.tif #getting Pong Dam dem
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/landsat2.zip
!unzip '*.zip'
!rm -r *.zip

In [ ]:
!pip install geopandas
!pip install rasterio

# Plotting tracks and solutions

In [ ]:
#@title **Plotting all files in combination of Track 1 and Track 2**{ vertical-output: true }
import glob
import itertools
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import h5py
import rasterio
from rasterio.plot import show
import numpy as np

dem_in = rasterio.open('/content/pongdam_dem.tif')
out_data = dem_in.read(1, masked=1)


# Define the track names
track_names = ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
################################################################################
def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  return h_ph_data, lat_ph_data, lon_ph_data
################################################################################
def plot_track_combinations(file1, file2, axs):
    # Loop through the track names and get the data
    data = []
    for i, tr in enumerate(track_names):
        try:
            h_ph_data1, lat1, lon1 = get_height_lat_lon(file1, tr)
            data.append((lon1, lat1, h_ph_data1, f"Track 1 {i+1} - {tr}"))
        except KeyError:
            print(f"Track {tr} not found in {file1}. Skipping...")

        try:
            h_ph_data2, lat2, lon2 = get_height_lat_lon(file2, tr)
            data.append((lon2, lat2, h_ph_data2, f"Track 2 {i+7} - {tr}"))
        except KeyError:
            print(f"Track {tr} not found in {file2}. Skipping...")

    # Find the bounding box that covers all tracks with a buffer
    min_lon, max_lon, min_lat, max_lat = float('inf'), float('-inf'), float('inf'), float('-inf')
    for lon, lat, _, _ in data:
        lon = np.array(lon)
        lat = np.array(lat)
        min_lon = min(np.min(lon), min_lon)
        max_lon = max(np.max(lon), max_lon)
        min_lat = min(np.min(lat), min_lat)
        max_lat = max(np.max(lat), max_lat)

    M_lon = (max_lon + min_lon)/2
    M_lat = (max_lat + min_lat)/2
    dx_boundary_x = max(max_lon - min_lon, max_lat - min_lat)

    # Set the limits of the axis to the bounding box
    axs.set_xlim(M_lon - dx_boundary_x/2, M_lon + dx_boundary_x/2)
    axs.set_ylim(M_lat - dx_boundary_x/2, M_lat + dx_boundary_x/2)

    show(dem_in, cmap='gist_ncar', ax=axs)

    # Plot each track separately with a label in the legend and annotate the label
    # Plot only the first point of each track with a label in the legend and annotate the label
    for lon, lat, h_ph_data, label in data:
        axs.scatter(lon, lat, s=1, marker='.', label=label)
        # axs.annotate(label, (lon[len(lon)/2], lat[len(lat)/2]), xytext=(5, 5), textcoords='offset points', fontsize=8)
        axs.annotate(label, (lon[len(lon)//2], lat[len(lat)//2]), xytext=(5, 5), textcoords='offset points', fontsize=8, bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))

    # Add a legend to the plot
    axs.legend(loc='lower right', scatterpoints=10)

    # Add axis labels and a title
    axs.set_xlabel('Longitude')
    axs.set_ylabel('Latitude')
    axs.set_title(f"Tracks 1-6 from {file1} \n and Tracks 7-12 from {file2}")

# Find all files with a .h5 extension in the current directory
files = glob.glob("*.h5")

combinations = set()
for f1, f2 in itertools.combinations(files, 2):
    if (f2, f1) not in combinations:
        combinations.add((f1, f2))

# Determine the number of possible combinations and set the number of rows and columns
num_combinations = len(list(combinations))
nrows = int(num_combinations**0.5) + 1
ncols = int(num_combinations/nrows) + 1

# Create a figure with subplots for each combination
fig, axs = plt.subplots(nrows, ncols, figsize=(100, 100))

for i, (file1, file2) in enumerate(combinations):
    row = i // ncols
    col = i % ncols
    index1 = row * nrows + col
    index2 = col * nrows + row
    print(f"({row},{col}): {file1} {file2}")
    plot_track_combinations(file1, file2, axs[row, col])
  

In [ ]:
import rasterio
from rasterio.plot import show

dem_in = rasterio.open('/content/pongdam_dem.tif')
out_data = dem_in.read(1, masked=1)

import numpy as np
import matplotlib.pyplot as plt
#@title **tracks buffer plot with its intersections at centre as well as height vs lat/lon of the buffered region**{ vertical-output: true }
fsize = 10
filename_1 = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
filename_2 = "/content/processed_ATL03_20200211111433_07140602_005_01.h5" #@param {type:"string"}
tr_1 = "gt1l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_2 = "gt1r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]


import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import jax.numpy as jnp
################################################################################
# Calculate distance along track using haversine formula
def haversine(lat1, lon1, lat2, lon2):
  R = 6371.0  # Earth radius in km
  dLat = np.radians(lat2 - lat1)
  dLon = np.radians(lon2 - lon1)
  lat1 = np.radians(lat1)
  lat2 = np.radians(lat2)
  a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
  c = 2*np.arcsin(np.sqrt(a))
  return R*c*1000.0
################################################################################
def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  return h_ph_data, lat_ph_data, lon_ph_data
################################################################################
h_ph_data1, lat1, lon1 = get_height_lat_lon(filename_1, tr_1)
h_ph_data2, lat2, lon2 = get_height_lat_lon(filename_2, tr_2)
################################################################################
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
pos = ax.get_position()
w_x, h_y = pos.width, pos.height
################################################################################plotting complete dem and track
lat_1x = np.array(lat1)
lon_1x = np.array(lon1)
lat_2x = np.array(lat2)
lon_2x = np.array(lon2)

# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1x), min(lon_2x)) - 0.001
max_lon = max(max(lon_1x), max(lon_2x)) + 0.001
min_lat = min(min(lat_1x), min(lat_2x)) - 0.001
max_lat = max(max(lat_1x), max(lat_2x)) + 0.001
M_lon = (max_lon + min_lon)/2
M_lat = (max_lat + min_lat)/2
dx_boundary_x = max(max_lon - min_lon, max_lat - min_lat)
# Set the limits of the axis to the bounding box
ax.set_xlim(M_lon - dx_boundary_x/2, M_lon + dx_boundary_x/2)
ax.set_ylim(M_lat - dx_boundary_x/2, M_lat + dx_boundary_x/2)
show(dem_in, cmap='gist_ncar', ax=ax)                            #plotting the dem over which the track points can be plotted
# Add a grid to the plot
ax.grid(True)

# Add a legend to the plot
# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title(f"Tracks 1 '{filename_1}' '{tr_1}'\n and \nTracks 2 '{filename_2}' '{tr_2}'")
################################################################################
lat_1 = np.array(lat1)
lon_1 = np.array(lon1)
lat_2 = np.array(lat2)
lon_2 = np.array(lon2)
################################################################################
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.geometry import LineString

# Convert the latitudes and longitudes to shapely Point objects
points_1 = [Point(lon, lat) for lat, lon in zip(lat_1, lon_1)]
points_2 = [Point(lon, lat) for lat, lon in zip(lat_2, lon_2)]

# Find the intersection of the two tracks
intersection = unary_union([LineString(points_1).intersection(LineString(points_2))])
buffer_region_in_meters = 1000*4 #@param {type:"raw"} # in meters
buffer_region = 1/(110 * 1000)* buffer_region_in_meters

# Create a buffer around the intersection point
intersection_buffer = intersection.buffer(buffer_region)
#@title **track in the buffered region with altitude/elevatoion/height buffer control for points visibility with track labels** { vertical-output: true }
# Compute the median height of the tracks within the buffered region
import geopandas as gpd

# Create a GeoDataFrame for the buffered intersection
intersection_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(intersection_buffer))

# Create a mask for selecting points within the buffered region
mask_1 = [intersection_buffer.contains(point) for point in points_1]
mask_2 = [intersection_buffer.contains(point) for point in points_2]

# Select the latitudes, longitudes, and heights within the buffered region for each track
lat_1_buf = lat_1[mask_1]
lon_1_buf = lon_1[mask_1]
height_1_buf = np.array(h_ph_data1)[mask_1]

lat_2_buf = lat_2[mask_2]
lon_2_buf = lon_2[mask_2]
height_2_buf = np.array(h_ph_data2)[mask_2]

##########################################################
# Set the limits of the axis to the bounding box
# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1_buf), min(lon_2_buf)) - 0.001
max_lon = max(max(lon_1_buf), max(lon_2_buf)) + 0.001
min_lat = min(min(lat_1_buf), min(lat_2_buf)) - 0.001
max_lat = max(max(lat_1_buf), max(lat_2_buf)) + 0.001
M_lon = (max_lon + min_lon)/2
M_lat = (max_lat + min_lat)/2
dx_boundary = max(max_lon - min_lon, max_lat - min_lat)
###########################################################
################################################################################
ax2 = fig.add_axes([0.5 -w_x/2 , 0-h_y , w_x , h_y])
ax2.set_position([pos.x0 , 0-h_y, pos.width, h_y])
#********************************
pos = ax2.get_position()
w_x, h_y = pos.width, pos.height
#********************************

x, y = intersection_buffer.exterior.xy

ax.fill(x, y, alpha=0.6, fc='#fa19b6', ec='#fcc603', linestyle='--', linewidth=1)
ax.scatter(lon_1x, lat_1x, s=1, color='blue', marker='.', label = 'Track 1')
ax.scatter(lon_2x, lat_2x, s=1, color='r', marker='.', label = 'Track 2')
ax.legend(loc='lower right', scatterpoints=10)

show(dem_in, cmap='gist_ncar', ax=ax2)                    #showing the dem on which the tracks will be  plotted

ax2.scatter(lon_1, lat_1,  marker='.', color='blue', s=1, label='Track 1')
ax2.scatter(lon_2, lat_2,  marker='.', color='red', s=1, label='Track 2')
ax2.fill(x, y, alpha=0.4, fc='#fa19b6', ec='#03fcc6', linestyle='dotted', linewidth=1)

ax2.set_xlim(M_lon - dx_boundary/2, M_lon + dx_boundary/2)
ax2.set_ylim(M_lat - dx_boundary/2, M_lat + dx_boundary/2)

ax2.grid(True)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Tracks 1 and 2 with Intersection Buffer')
ax2.legend(['Track 1', 'Track 2'], loc='lower right', scatterpoints=10)
################################################################################< elevation vs latitude
median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))
buffer_height = 100 #@param {type:"raw"}

ax3 = fig.add_axes([0.5 -w_x/2 , 0-h_y , w_x , h_y], sharey=ax2)
ax3.set_position([pos.x0 + pos.width , pos.y0 , w_x/3, pos.height])

ax3.scatter(height_1_buf, lat_1_buf,  marker='.', color='blue', s=1, label='Track 1')
ax3.scatter(height_2_buf, lat_2_buf,  marker='.', color='red', s=1, label='Track 2')
ax3.set_xlim(M_lat - dx_boundary/2, M_lat + dx_boundary/2)
ax3.axvspan(median_height - buffer_height, median_height + buffer_height, color='#fdffa3', alpha=0.3)
ax3.yaxis.set_label_position('right')
ax3.yaxis.tick_right()
ax3.set_ylabel('Y-axis label', visible=False)
ax3.set_xlim([median_height - buffer_height, median_height + buffer_height])
ax3.set_xlabel('Height')
ax3.set_ylabel('Latitude')
ax3.set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')
ax3.grid(True,  color='black', alpha = 0.9)
ax3.legend(loc='lower right', scatterpoints=10)
################################################################################< elevation vs longitude
ax4 = fig.add_axes([0 , 0 , w_x , h_y/3], sharex=ax2)
ax4.set_position([pos.x0 , pos.y0 - h_y/3 -0.1, pos.width, h_y/3])

ax4.scatter(lon_1_buf, height_1_buf,  marker='.', color='blue', s=1, label='Track 1')
ax4.scatter(lon_2_buf, height_2_buf,  marker='.', color='red', s=1, label='Track 2')
ax4.set_xlim(M_lon - dx_boundary/2, M_lon + dx_boundary/2)
ax4.axhspan(median_height - buffer_height, median_height + buffer_height, color='#fdffa3', alpha=0.3)
ax4.set_ylim([median_height - buffer_height, median_height + buffer_height])
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Height')
ax4.set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region', loc='center').set_position([.5, 1.01])
ax4.grid(True,  color='black', alpha = 0.9)
ax4.legend(loc='lower right', scatterpoints=10)
################################################################################< elevation vs track 1 & 2 distances
ax5 = fig.add_axes([0.5 - w_x/2 , -h_y - h_y/3 -h_y/3 , w_x , h_y/3])
ax5.set_position([pos.x0 ,-h_y - h_y/3 -h_y/3 - 0.1 * 2, pos.width, h_y/3])

dist_ph_along_data_1 = [haversine(lat_1_buf[0], lon_1_buf[0], lat_1_buf[i], lon_1_buf[i]) for i in range(len(lat_1_buf))]

lats = np.flip(lat_2_buf)
longs = np.flip(lon_2_buf)
heights = np.flip(height_2_buf)

dist_ph_along_data_2 = [haversine(lats[0], longs[0], lats[i], longs[i]) for i in range(len(lats))]
ax5.scatter(np.array(dist_ph_along_data_1), np.array(height_1_buf),  marker= '.', s=1, alpha= 0.4, label = "Track 1")
ax5.scatter(np.array(dist_ph_along_data_2), np.array(heights),  marker= '.', s=1, alpha= 0.4, label = "Track 2")

ax5.set_ylim([median_height - buffer_height, median_height + buffer_height])
ax5.set_xlabel('Along-Track Distance(m)')
ax5.set_ylabel('photon height')
ax5.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights', loc='center')
ax5.grid(True,  color='black', alpha = 0.9)
ax5.legend(loc='lower right', scatterpoints=10)

# Show the plot
plt.subplots_adjust(hspace=0)
plt.show()


testing ellipsiod height to orthometruc height conversion

# transform

**once we get the `lat_1_buf`, `lon_1_buf`& `height_1_buf` then we can save it in *Point cloud* numpy format as `point_cloud.npy` for the purpose of point cloud classification and other things**

In [ ]:
len(np.array(lat_1_buf))

In [ ]:
import numpy as np

# Define your lat, lon, and height arrays
lat_array = lat_1_buf
lon_array = lon_1_buf
height_array = height_1_buf

# Combine the arrays into a single NumPy array
point_cloud_array = np.column_stack((lon_array, lat_array, height_array))

# Save the NumPy array to a file using np.save()
np.save('point_cloud.npy', point_cloud_array)


testing graph cut

In [ ]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt

# Generate 100 random points in a 2D plane
points = np.random.rand(100, 2)

# Compute Voronoi diagram
vor = Voronoi(points)

# Plot Voronoi diagram
fig, ax = plt.subplots()
voronoi_plot_2d(vor, ax=ax)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dLat, dLon = np.radians(lat2 - lat1), np.radians(lon2 - lon1)
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R*c*1000.0

# Load data from .npy file
data = np.load('/content/point_cloud.npy')

# Extract latitude, longitude, and elevation data
lon, lat, elev = data[:,0], data[:,1], data[:,2]

# Calculate linear distance between points
dist = np.zeros_like(lat)
for i in range(1, len(lat)):
    dist[i] = dist[i-1] + haversine(lat[i-1], lon[i-1], lat[i], lon[i])

# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 10), dpi = 100)
ax.scatter(dist, elev, s= 1)
ax.set_aspect('equal')
ax.set_xlabel('Linear Distance (m)')
ax.set_ylabel('Elevation (m)')
ax.grid(True)
plt.show()


In [ ]:

interval=100
spacing=30
ncols=10
start_dist=0
dist_intervals = []
for i in range(len(dist)):
    if dist[i] % interval == 0 and dist[i] >= start_dist:
        dist_intervals.append((dist[i], dist[i] + interval))
    elif dist[i] % interval >= spacing and dist[i] >= start_dist:
        dist_intervals.append((dist[i], dist[i] + interval))
num_intervals = len(dist_intervals)
elev_intervals = [np.array([]) for i in range(num_intervals)]
for i in range(num_intervals):
    mask = (dist_intervals[i][0] <= dist) & (dist < dist_intervals[i][0] + interval) & (~np.isnan(elev))
    elev_intervals[i] = elev[mask]
    if (len(elev_intervals[i]) == 1):
      elev_intervals[i] = elev_intervals[i - 1]

def convert_data(dist_intervals, elev_intervals):
  """
  Converts the data variable to take input as `(dist_intervals[:][0], elev_intervals[:][0])` distances and elevation arrays.

  Args:
    dist_intervals: The distances array.
    elev_intervals: The elevation array.

  Returns:
    The converted data variable.
  """

  # Create a copy of the data variable.
  data = dist_intervals.copy()

  # Loop over the distances array.
  for i in range(len(dist_intervals)):
    # Get the current distance interval.
    current_dist_interval = dist_intervals[i]

    # Get the current elevation interval.
    current_elev_interval = elev_intervals[i]

    # Set the value of the data variable at the current index to the product of the current distance interval and the current elevation interval.
    data[i] = current_dist_interval * current_elev_interval

  return data


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some data with outliers.
# data = elev_intervals[:][0]
data = elev
dist_a = dist
print(len(data))
#hampel filter
def hampel_filter(data, window_size, k):
  filtered_data = data.copy()
  for i in range(len(data) - window_size + 1):
    median = np.median(data[i:i + window_size])
    mad = np.median(np.abs(data[i:i + window_size] - median))
    threshold = k * mad
    for j in range(window_size):
      if np.abs(data[i + j] - median) > threshold:
        filtered_data[i + j] = median

  return filtered_data

# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 5), dpi = 100)
ax.scatter(dist, elev, s= 1)
ax.set_aspect('equal')
ax.set_xlabel('Linear Distance (m)')
ax.set_ylabel('Elevation (m)')
ax.grid(True)
# Create scatter plot
fig, ax1 = plt.subplots(figsize=(10, 5), dpi = 100)
ax1.scatter(dist_a, data, s= 1)
ax1.set_ylim(np.median(data)-10, np.median(data)+100)
# ax1.set_aspect('equal')
ax1.set_xlabel('Linear Distance (m)')
ax1.set_ylabel('Elevation (m)')
ax1.grid(True)
# Filter the data using the Hampel filter with a window size of 5 and a threshold of 3.
filtered_data = hampel_filter(data, 17, 3)
print(len(filtered_data))

from scipy.signal import medfilt
# # Apply median filter with window size of 3 to filtered data
median_filtered_data = medfilt(filtered_data, kernel_size=17)
start = 0
end = 40000 +start
# Add a smooth moving average over the median filtered data.
smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')
for i in range(0,4):
  filtered_data = hampel_filter(median_filtered_data, 17, 3)
  filtered_data = hampel_filter(filtered_data, 17, 3)
  filtered_data = hampel_filter(filtered_data, 17, 3)
  median_filtered_data = medfilt(filtered_data, kernel_size=17)
  median_filtered_data = medfilt(median_filtered_data, kernel_size=17)

smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')

# dist_a_avg = np.convolve(dist_a, np.ones(5)/5, mode='same')
print(len(dist_a),len(smooth_moving_average), len(dist_a_avg))
# Plot the original data, the Hampel filtered data, the median filtered data, and the smooth moving average.
fig, ax = plt.subplots(figsize=(10, 5), dpi = 300)
ax.scatter(dist_a[start:end], data[start:end], label="Original data", s=1)
ax.scatter(dist_a[start:end], filtered_data[start:end], label="Hampel filtered data", s=1)
ax.scatter(dist_a[start:end], median_filtered_data[start:end], label="Median filtered data", s = 1)
ax1.scatter(dist_a[start:end], median_filtered_data[start:end], label="Median filtered data", s = 1, c = "r")
ax.scatter(dist_a[start:end], smooth_moving_average[start:end], label="Smooth moving average", s = 1)
ax.set_ylim(np.median(median_filtered_data[start:end])-100, np.median(median_filtered_data[start:end])+100)

# Add annotations for the beginning and end points.
ax.annotate(f"{smooth_moving_average[start]:.2f}", xy=(dist_a_avg[start], smooth_moving_average[start]), xytext=(-20, 10), textcoords='offset points')
ax.annotate(f"{smooth_moving_average[end-1]:.2f}", xy=(dist_a_avg[end-1], smooth_moving_average[end-1]), xytext=(-20, 10), textcoords='offset points')

# ax.set_aspect('equal')
ax.set_xlabel('Linear Distance (m)')
ax.set_ylabel('Elevation (m)')
ax.legend()

plt.show()


In [ ]:
data = elev #elevation array
dist_a = dist #linear distance array

#hampel filter
def hampel_filter(data, window_size, k):

  # Create a copy of the data.
  filtered_data = data.copy()

  # Loop over the data, one window at a time.
  for i in range(len(data) - window_size + 1):
    # Calculate the median of the data within the window.
    median = np.median(data[i:i + window_size])

    # Calculate the MAD of the data within the window.
    mad = np.median(np.abs(data[i:i + window_size] - median))

    # Calculate the threshold for outliers.
    threshold = k * mad

    # If any of the data points in the window are outliers, replace them with the median.
    for j in range(window_size):
      if np.abs(data[i + j] - median) > threshold:
        filtered_data[i + j] = median

  return filtered_data


# Filter the data using the Hampel filter with a window size of 5 and a threshold of 3.
filtered_data = hampel_filter(data, 5, 30)
print(len(filtered_data))

from scipy.signal import medfilt
# # Apply median filter with window size of 3 to filtered data
median_filtered_data = medfilt(filtered_data, kernel_size=7)

# Add a smooth moving average over the median filtered data.
smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def slope_detector(points, window_size, threshold):
    slopes = []
    for i in range(window_size, len(points)):
        window_points = points[i-window_size:i]
        x = np.arange(window_size)
        y = window_points
        m, b = np.polyfit(x, y, 1)  # find the slope of the line
        slopes.append(m)
    
    # Find the points with slope greater than the threshold
    idxs = np.where(np.abs(slopes) > threshold)[0]
    differentiated_points = np.zeros_like(points)
    differentiated_points[idxs + window_size//2] = points[idxs + window_size//2]
    
    return differentiated_points

# Example usage
points = smooth_moving_average[:]
differentiated_points = slope_detector(points, window_size=17, threshold=0.04)
plt.subplots(figsize=(10, 3), dpi = 100)
plt.plot(points, 'b-', label='Original Points')
plt.plot(differentiated_points, 'ro', label='Differentiated Points')
# plt.gca().set_aspect('equal')
plt.ylim(np.median(points)-10, np.median(points)+40)
plt.legend()
plt.show()


In [ ]:
import numpy as np

def slope_detector(points, window_size, threshold):
    slopes = []
    for i in range(window_size, len(points)):
        window_points = points[i-window_size:i]
        x = np.arange(window_size)
        y = window_points
        m, b = np.polyfit(x, y, 1)  # find the slope of the line
        slopes.append(m)
    
    # Find the points with slope greater than the threshold
    idxs = np.where(np.abs(slopes) > threshold)[0]
    inflection_points = np.column_stack((idxs + window_size//2, points[idxs + window_size//2]))
    
    return inflection_points

import matplotlib.pyplot as plt

points = smooth_moving_average[:]
window_size = 5
threshold = 0.5

inflection_points = slope_detector(points, window_size= 17, threshold= 0.04)
plt.plot(points, 'b-', label='Original Points')

plt.scatter(inflection_points[:, 0], inflection_points[:, 1])
plt.show()


In [ ]:
unique_years

In [ ]:
import datetime
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# Extract dates from list_C using regex
dates = [datetime.datetime.strptime(re.search(r'processed_ATL03_(\d{8})', path).group(1), "%Y%m%d").date() for path in list_C]

# Convert dates to matplotlib dates
mpl_dates = mdates.date2num(dates)

# Extract unique years from dates
unique_years = np.unique([date.year for date in dates])

# Define the color map
cmap = plt.cm.get_cmap('tab20')

# Plot the data for each year
fig, ax = plt.subplots(figsize=(4, 6))
for i, year in enumerate(unique_years):
  # Filter the dates and y-values to plot
  mask = np.array([date.year == year for date in dates])
  mpl_dates_filtered = mpl_dates[mask]
  # Modify rand_y_values to contain days of the month
  rand_x_values = [date.day for date in np.array(dates)[mask].tolist()]
  # Assign a color to the year to plot
  color = cmap(i/len(unique_years))

  # Plot the data
  ax.scatter(rand_x_values, mpl_dates_filtered, c=color, label=year)


# Format the x-axis and y-axis
ax.yaxis.set_major_locator(mdates.MonthLocator())
ax.yaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax.tick_params(axis='y', labelrotation=0)
ax.set_xlabel('X-Values')

# Add a legend
ax.legend(title='Years')

# Show the plot
plt.show()


In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
filename_1 = "/content/processed_ATL03_20210131181032_05921002_005_01.h5" #@param {type:"string"}
tr_1 = "gt2r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]

# Define haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dLat, dLon = np.radians(lat2 - lat1), np.radians(lon2 - lon1)
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R*c*1000.0
def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  return h_ph_data, lat_ph_data, lon_ph_data
  

h_ph_data1, lat1, lon1 = get_height_lat_lon(filename_1, tr_1)
# Combine the arrays into a single NumPy array
point_cloud_array = np.column_stack((lon1, lat1, h_ph_data1))
np.save('point_cloud.npy', point_cloud_array)



In [ ]:
# Load data from .npy file
data = np.load('/content/point_cloud.npy')

# Extract latitude, longitude, and elevation data
lon, lat, elev = data[:,0], data[:,1], data[:,2]

In [ ]:


# Load data from .npy file
data = np.load('/content/point_cloud.npy')

# Extract latitude, longitude, and elevation data
lon, lat, elev = data[:,0], data[:,1], data[:,2]

# Calculate linear distance between points
dist = np.zeros_like(lat)
for i in range(1, len(lat)):
    dist[i] = dist[i-1] + haversine(lat[i-1], lon[i-1], lat[i], lon[i])


import numpy as np
import matplotlib.pyplot as plt

# Generate some data with outliers.
# data = elev_intervals[:][0]
data = elev
dist_a = dist
print(len(data))
#hampel filter
def hampel_filter(data, window_size, k):
  filtered_data = data.copy()
  for i in range(len(data) - window_size + 1):
    median = np.median(data[i:i + window_size])
    mad = np.median(np.abs(data[i:i + window_size] - median))
    threshold = k * mad
    for j in range(window_size):
      if np.abs(data[i + j] - median) > threshold:
        filtered_data[i + j] = median

  return filtered_data

# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 5), dpi = 100)
ax.scatter(dist, elev, s= 1)
ax.set_aspect('equal')
ax.set_xlabel('Linear Distance (m)')
ax.set_ylabel('Elevation (m)')
ax.grid(True)
# Create scatter plot
fig, ax1 = plt.subplots(figsize=(10, 5), dpi = 100)
ax1.scatter(dist_a, data, s= 1)
ax1.set_ylim(np.median(data)-10, np.median(data)+100)
# ax1.set_aspect('equal')
ax1.set_xlabel('Linear Distance (m)')
ax1.set_ylabel('Elevation (m)')
ax1.grid(True)

# Filter the data using the Hampel filter with a window size of 5 and a threshold of 3.
filtered_data = hampel_filter(data, 17, 3)
print(len(filtered_data))

from scipy.signal import medfilt
# # Apply median filter with window size of 3 to filtered data
median_filtered_data = medfilt(filtered_data, kernel_size=17)
start = 0
end = 2000 +start
# Add a smooth moving average over the median filtered data.
smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')

# Plot the original data, the Hampel filtered data, the median filtered data, and the smooth moving average.
fig, ax = plt.subplots(figsize=(10, 5), dpi = 300)
ax.scatter(dist_a[start:end], data[start:end], label="Original data", s=1)
ax.scatter(dist_a[start:end], filtered_data[start:end], label="Hampel filtered data", s=1)
ax.scatter(dist_a[start:end], median_filtered_data[start:end], label="Median filtered data", s = 1)
ax1.scatter(dist_a[start:end], median_filtered_data[start:end], label="Median filtered data", s = 1, c = "r")
ax.scatter(dist_a[start:end], smooth_moving_average[start:end], label="Smooth moving average", s = 1)
ax.set_ylim(np.median(median_filtered_data[start:end])-100, np.median(median_filtered_data[start:end])+100)

# # Add annotations for the beginning and end points.
# ax.annotate(f"{smooth_moving_average[start]:.2f}", xy=(dist_a_avg[start], smooth_moving_average[start]), xytext=(-20, 10), textcoords='offset points')
# ax.annotate(f"{smooth_moving_average[end-1]:.2f}", xy=(dist_a_avg[end-1], smooth_moving_average[end-1]), xytext=(-20, 10), textcoords='offset points')

# ax.set_aspect('equal')
ax.set_xlabel('Linear Distance (m)')
ax.set_ylabel('Elevation (m)')
ax.legend()

plt.show()


# preparation for **plotting in river buffer**

In [ ]:
#downloading river buffer
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/parvati_catchment.zip
!unzip parvati_catchment.zip -d parvati_catchment
#downloading catchment
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/parvati_river_buffer.zip
!unzip parvati_river_buffer.zip -d parvati_river_buffer
#parallel downloading of ICESAT 2 tracks
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/parvati_tracks.txt
import multiprocessing as mp
import subprocess
import re
# Define the regex pattern to match the links
pattern = r"https://n5eil02u.ecs.nsidc.org/esir/.*?\.h5"
with open("parvati_tracks.txt", "r") as file:
    contents = file.read()
list_C = re.findall(pattern, contents)

# Define the function to download a file using wget
def download_file(url):
    subprocess.Popen(['wget', '--http-user=kroy0001', '--http-password=/#j%kWrPA,8.HRe', url]).wait()

# Create a pool of processes to download the files
num_processes = mp.cpu_count() * 2
pool = mp.Pool(processes=num_processes)
results = [pool.apply_async(download_file, args=(url,)) for url in list_C]

# Wait for all the processes to finish
for r in results:
    r.wait()

**funny day**

In [ ]:
!pip install geopandas

In [ ]:
#@title filing { vertical-output: true }
import h5py
import numpy as np
import os
import glob
import re
import datetime
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Load the vector file
vector_file = "/content/parvati_river_buffer/Export_Output.shp"
region = gpd.read_file(vector_file)

def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  file_name = np.array([filename] * len(h_ph_data))
  track_name = np.array([track_name] * len(h_ph_data))
  return np.column_stack((file_name, track_name, lon_ph_data, lat_ph_data, h_ph_data))

# Define the track names to be checked
track_names = ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]

h5_files = glob.glob("*.h5")
file_list = []

for filename in h5_files:
  try:
    f = h5py.File(filename, 'r')
    for track_name in track_names:
      np.array(f['/'+str(track_name)+'/heights/h_ph'])
      file_list.append((filename, track_name)) # Append (filename, track_name) tuple to file_list
  except KeyError:
    count = 0

clipped_points_array = []

h5_files = file_list
count = 0
def process_data_by_year(h5_files, year, region):
    for filename, track in tqdm(h5_files):
        date_string = re.search(r'processed_ATL03_(\d{8})', filename).group(1)
        date_object = datetime.datetime.strptime(date_string, "%Y%m%d").date().year
        if date_object == year:
            try:
                track_data = get_height_lat_lon(filename, track)
                np.save(f'point_cloud_{filename[:-3]}_{track}.npy', track_data)
                point_cloud_file = f'point_cloud_{filename[:-3]}_{track}.npy'
                point_cloud = np.load(point_cloud_file, allow_pickle=True)
                file_names, track_names, lon, lat, h_ph_data = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2], point_cloud[:, 3], point_cloud[:, 4]
                geometry = [Point(lon[i], lat[i]) for i in range(len(lon))]
                data = gpd.GeoDataFrame({'file_name': file_names, 'track_name': track_names,'h_ph_data': h_ph_data, 'lat': lat, 'lon': lon},geometry=geometry, crs=region.crs)
                clipped_points = gpd.sjoin(data, region, op='within')
                clipped_points_array.append(np.column_stack((clipped_points['file_name'], clipped_points['track_name'], clipped_points['h_ph_data'], clipped_points['lat'], clipped_points['lon'])))

                fig, ax = plt.subplots(figsize=(10, 6))
                region.plot(ax=ax, color='white', edgecolor='black')
                data.plot(ax=ax, column='h_ph_data', markersize=1)

                plt.title("Points inside the Buffered Vector")
                plt.legend()
                plt.show()
            except KeyError:
                count += 1
year_list = [2018, 2019, 2020, 2021, 2022]
for x in year_list:
  process_data_by_year(h5_files, x, region)

np.save('clipped_points.npy', clipped_points_array)


In [ ]:
#@title **plotting functions**
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt

# Define haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dLat, dLon = np.radians(lat2 - lat1), np.radians(lon2 - lon1)
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R*c*1000.0


def elevation_plot(data, window_size=17, k=3):
    # Load data from .npy file
    # data = np.load(np_file, allow_pickle=True)

    # Extract latitude, longitude, and elevation data
    lon, lat, elev = data[:,4].astype(float), data[:,3].astype(float), data[:,2].astype(float)
    # Calculate linear distance between points
    dist = np.zeros_like(lat)
    for i in range(1, len(lat)):
        dist[i] = dist[i-1] + haversine(lat[i-1], lon[i-1], lat[i], lon[i])

    # Apply Hampel filter with specified window size and threshold
    filtered_data = hampel_filter(elev, window_size, k)

    # Apply median filter with window size of 17 to filtered data
    median_filtered_data = medfilt(filtered_data, kernel_size=17)

    # Add a smooth moving average over the median filtered data.
    smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')

    # for i in range(0,10):
    #   filtered_data = hampel_filter(smooth_moving_average, window_size, k)
    #   median_filtered_data = medfilt(filtered_data, kernel_size=17)
    #   smooth_moving_average = np.convolve(median_filtered_data, np.ones(5)/5, mode='same')

    # Create scatter plot
    fig, ax = plt.subplots(figsize=(10, 5), dpi=300)
    ax.scatter(dist, elev, label="Original data", s=1)
    ax.scatter(dist, filtered_data, label="Hampel filtered data", s=1)
    ax.scatter(dist, median_filtered_data, label="Median filtered data", s=1)
    ax.scatter(dist, smooth_moving_average, label="Smooth moving average", s=1, c='r')
    ax.set_aspect('equal')
    ax.set_xlabel('Linear Distance (m)')
    ax.set_ylabel('Elevation (m)')
    ax.set_title('Elevation Profile '+data[0,0]+' '+data[0,1])
    ax.legend()

# Hampel filter
def hampel_filter(data, window_size, k):
    filtered_data = data.copy()
    for i in range(len(data) - window_size + 1):
        median = np.median(data[i:i + window_size])
        mad = np.median(np.abs(data[i:i + window_size] - median))
        threshold = k * mad
        for j in range(window_size):
            if np.abs(data[i + j] - median) > threshold:
                filtered_data[i + j] = median

    return filtered_data


In [ ]:
point_cloud_file = f'/content/clipped_points.npy'
P_C = np.load(point_cloud_file, allow_pickle=True)
PC_list =[]
for point_cloud in P_C:
  if len(point_cloud) != 0:
    PC_list.append(point_cloud)

fig, ax = plt.subplots(figsize=(10,5))
region.plot(ax=ax, color='white', edgecolor='black')

for point_cloud in PC_list:
  # Extract the file names, track names, and point cloud data from the numpy array
  file_names = point_cloud[:, 0]
  track_names = point_cloud[:, 1]  
  h_ph_data = point_cloud[:, 2]
  lat = point_cloud[:, 3]
  lon = point_cloud[:, 4]

  # Create a GeoDataFrame for the point cloud data
  geometry = [Point(lon[i], lat[i]) for i in range(len(lon))]
  data = gpd.GeoDataFrame({'file_name': file_names, 'track_name': track_names,'h_ph_data': h_ph_data, 'lat': lat, 'lon': lon}, geometry=geometry, crs=region.crs)

  # Spatially join the points with the region to get the clipped points
  clipped_points = gpd.sjoin(data, region, op='within')
  clipped_points.plot(ax=ax, column='h_ph_data', markersize=1)

  first_points = clipped_points.groupby('track_name').first()
  for idx, row in first_points.iterrows():
    point = row['geometry']
    label = row['file_name']
    ax.annotate(label, xy=(point.x, point.y), xytext=(3, 3), textcoords="offset points", fontsize=8, color='red')
    
  elevation_plot(point_cloud, window_size=17, k=3)

plt.show()